In [1]:
import pandas as pd

In [136]:
df = pd.read_csv('allshootings.csv')
df.head(10)

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,2509362,"January 23, 2023",Illinois,Chicago,2900 block of East 78th St,2,3,NaN
1,2509663,"January 23, 2023",California,Half Moon Bay,12761 San Mateo Rd,7,1,NaN
2,2508683,"January 22, 2023",Mississippi,Robinsonville (Tunica Resorts),1010 Casino Center Dr,0,4,NaN
3,2508370,"January 22, 2023",Louisiana,Baton Rouge,4619 Bennington Ave,0,12,NaN
4,2508522,"January 22, 2023",Louisiana,Shreveport,1600 Sugar Ln,0,8,NaN
5,2507836,"January 21, 2023",California,Monterey Park,122 W Garvey Ave,12,9,NaN
6,2504645,"January 17, 2023",Texas,Houston,6729 Lockwood Dr,0,4,NaN
7,2503649,"January 16, 2023",Florida,Fort Pierce,Avenue M and N 13th St,1,7,NaN
8,2503345,"January 16, 2023",California,Goshen,Harvest Ave and Rd 68,6,0,NaN
9,2503245,"January 16, 2023",Florida,Sanford,Rinehart Rd and Co Rd 46A,1,5,NaN


In [137]:
total_killed = df['# Killed'].sum()
total_killed

1968

In [138]:
total_injured = df['# Injured'].sum()
total_injured

8298

In [139]:
df['Incident Date'] = pd.to_datetime(df['Incident Date'])

In [16]:

import plotly.express as px
plot = px.scatter(df, x = 'Incident Date', y = '# Killed', color = 'State', title = '')
plot.show()

In [35]:
sum_df = df.groupby(['State'], as_index = False).sum().sort_values('# Killed')
sum_df.head(10)

,State,Incident ID,# Killed,# Injured,Operations
27,New Hampshire,2064260,0,4,0.0
36,Rhode Island,4083870,0,13,0.0
18,Maine,1613020,3,1,0.0
29,New Mexico,16278770,4,32,0.0
38,South Dakota,3884272,4,5,0.0
11,Idaho,3775827,5,6,0.0
15,Kansas,14875199,5,29,0.0
1,Alaska,7378729,6,12,0.0
20,Massachusetts,29348275,6,65,0.0
7,Delaware,24208940,7,44,0.0


In [42]:
sum_df['Killed_or_Injured'] = sum_df['# Killed'] + sum_df['# Injured']
sum_df = sum_df.sort_values('Killed_or_Injured')

In [67]:
avg_df = df.groupby('State', as_index=False).mean()
avg_df = avg_df.sort_values('# Killed')

In [98]:
hovertemplate = "# Killed: %{customdata[0]} <br># Injured: %{customdata[1]}: <br># Killed or Injured: %{customdata[2]} </br>"
plot = px.scatter(sum_df, x = 'State', y = 'Killed_or_Injured', color = 'State', title = 'Sorted Mass Shootings by Killed or Injured from 2019 - 2023', custom_data = ['# Killed', '# Injured', 'Killed_or_Injured'])
plot.update_traces(hovertemplate = hovertemplate)
plot.show()
plot.write_html('shootings.html')

In [57]:
sum_df = sum_df.sort_values('# Killed')
hovertemplate = "# Killed: %{customdata[0]} <br># Injured: %{customdata[1]}: <br># Killed or Injured: %{customdata[2]} </br>"
plot = px.scatter(sum_df, x = 'State', y = '# Killed', color = 'State', title = 'Mass Shootings from 2019 - 2023', custom_data = ['# Killed', '# Injured', 'Killed_or_Injured'], trendline='ols')
plot.update_traces(hovertemplate = hovertemplate)
plot.show()
plot.write_html('shootings_sorted_killed.html')

In [71]:
avg_df = avg_df.round(2)
hovertemplate = "# Killed: %{customdata[0]} <br># Injured: %{customdata[1]}: </br>"
plot = px.scatter(avg_df, x = 'State', y = '# Killed', color = 'State', title = 'Avg Killed in shootings from 2019 - 2023', custom_data = ['# Killed', '# Injured'], trendline='ols')
plot.update_traces(hovertemplate = hovertemplate)
plot.show()

In [76]:
dic = {}
for state in df['State']:
    dic[state] = len(df[df['State'] == state])
dic

{'Illinois': 216,
 'California': 140,
 'Mississippi': 45,
 'Louisiana': 92,
 'Texas': 145,
 'Florida': 103,
 'Arizona': 24,
 'Missouri': 54,
 'Ohio': 78,
 'Pennsylvania': 105,
 'Minnesota': 32,
 'Colorado': 38,
 'Georgia': 70,
 'Alabama': 51,
 'North Carolina': 66,
 'Utah': 3,
 'Maryland': 59,
 'Virginia': 49,
 'District of Columbia': 34,
 'Tennessee': 51,
 'New York': 106,
 'Kansas': 7,
 'Oklahoma': 12,
 'Michigan': 72,
 'Nebraska': 8,
 'New Jersey': 35,
 'South Carolina': 59,
 'Wisconsin': 37,
 'Massachusetts': 15,
 'Arkansas': 23,
 'Oregon': 15,
 'Kentucky': 23,
 'Washington': 23,
 'Indiana': 36,
 'Nevada': 14,
 'New Mexico': 8,
 'Delaware': 12,
 'Connecticut': 12,
 'Iowa': 12,
 'South Dakota': 2,
 'Idaho': 2,
 'West Virginia': 4,
 'Rhode Island': 2,
 'New Hampshire': 1,
 'Alaska': 4,
 'Maine': 1}

In [94]:
number_df = pd.DataFrame([dic])
number_df = number_df.T
number_df = number_df.rename(columns = {0: 'Number_Of_Shootings'})
number_df.sort_values('Number_Of_Shootings', ascending=False)
number_df.to_csv('output.csv')

In [95]:
number_df = pd.read_csv('output.csv')

In [96]:
sum_df = pd.merge(sum_df, number_df, on='State')

In [140]:
# SCORE FUNC
df['Danger Score'] = df['# Killed']*2 + df['# Injured'] + # number of shootings / population + population density + drop out rates 
df

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations,Danger Score
0,2509362,2023-01-23,Illinois,Chicago,2900 block of East 78th St,2,3,NaN,7
1,2509663,2023-01-23,California,Half Moon Bay,12761 San Mateo Rd,7,1,NaN,15
2,2508683,2023-01-22,Mississippi,Robinsonville (Tunica Resorts),1010 Casino Center Dr,0,4,NaN,4
3,2508370,2023-01-22,Louisiana,Baton Rouge,4619 Bennington Ave,0,12,NaN,12
4,2508522,2023-01-22,Louisiana,Shreveport,1600 Sugar Ln,0,8,NaN,8
...,...,...,...,...,...,...,...,...,...
1995,1579266,2019-12-24,Louisiana,New Orleans,6400 block of General Meyer Ave,0,4,NaN,4
1996,1594317,2019-12-24,Illinois,Joliet,1200 block of Cutter Ave,0,5,NaN,5
1997,1579220,2019-12-24,North Carolina,High Point,1226 Montlieu Ave,0,6,NaN,6
1998,1577577,2019-12-22,Maryland,Baltimore,225 Park Ave,0,7,NaN,7


In [141]:
df = df.groupby('State', as_index=False).sum()
# df = df.drop(columns=['Number_Of_Shootings_x', 'Number_Of_Shootings_y', 'Number_Of_Shootings_x', 'Number_Of_Shootings_y'])
df = pd.merge(df, number_df, on='State')
df.head(10)

,State,Incident ID,# Killed,# Injured,Operations,Danger Score,Number_Of_Shootings
0,Alabama,106757836,51,200,0.0,302,51
1,Alaska,7378729,6,12,0.0,24,4
2,Arizona,52341657,34,110,0.0,178,24
3,Arkansas,44639319,21,110,0.0,152,23
4,California,290120389,185,531,0.0,901,140
5,Colorado,78915508,62,154,0.0,278,38
6,Connecticut,23086778,8,46,0.0,62,12
7,Delaware,24208940,7,44,0.0,58,12
8,District of Columbia,69908507,25,158,0.0,208,34
9,Florida,212171091,83,463,0.0,629,103


In [134]:
df = df.sort_values('Danger Score')
df.head(10)

,State,Incident ID,# Killed,# Injured,Operations,Danger Score,Number_Of_Shootings_x,Number_Of_Shootings_y,Number_Of_Shootings_x,Number_Of_Shootings_y
4,New Hampshire,2064260,0,4,0.0,4,1,1,1,1
3,Maine,1613020,3,1,0.0,7,1,1,1,1
6,Rhode Island,4083870,0,13,0.0,13,2,2,2,2
7,South Dakota,3884272,4,5,0.0,13,2,2,2,2
1,Idaho,3775827,5,6,0.0,16,2,2,2,2
0,Alaska,7378729,6,12,0.0,24,4,4,4,4
8,Utah,6121692,13,5,0.0,31,3,3,3,3
9,West Virginia,7471958,12,9,0.0,33,4,4,4,4
2,Kansas,14875199,5,29,0.0,39,7,7,7,7
5,New Mexico,16278770,4,32,0.0,40,8,8,8,8


In [190]:
df['Danger Score Normalized'] = df['Danger Score'] /df['Danger Score'].abs().max()
df = df.round(2)

In [191]:
hovertemplate = "State: %{customdata[4]} <br>Killed: %{customdata[0]} <br>Injured: %{customdata[1]}: <br>Number of Shootings: %{customdata[2]} <br>Score (x2 for Killed, x1 for Injured): %{customdata[3]} </br>"
plot = px.scatter(df, x = 'State', y = 'Danger Score', color = 'Number_Of_Shootings', title = 'Danger Score in shootings from 2019 - 2023', custom_data = ['# Killed', '# Injured', 'Number_Of_Shootings', 'Danger Score Normalized', 'State'])
plot.update_traces(hovertemplate = hovertemplate)
plot.show()
plot.write_html('DangerScore.html')

In [174]:
len(df['State'])

46

In [179]:
states = """Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming"""
states_arr = states.split('\n')


In [183]:
no_shooting = []
for state in states_arr:
    if len(df[df['State'] == state]) > 0:
        continue
    else:
        no_shooting.append(state)

no_shooting

['Hawaii', 'Montana', 'North Dakota', 'Vermont', 'Wyoming']